In [4]:
import nltk
import numpy as np
import operator as op
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#chargement des donnes
liste_id = []
liste_label = []
liste_message = []
file_train = '/content/drive/My Drive/Analyse_des_sentiment/data/train.xml'
file_dev   = '/content/drive/My Drive/Analyse_des_sentiment/data/dev.xml'
file_test  = '/content/drive/My Drive/Analyse_des_sentiment/data/test.xml'

In [7]:
def read_xml_to_dataframe(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        movie = comment.find('movie').text
        review_id = comment.find('review_id').text
        name = comment.find('name').text
        user_id = comment.find('user_id').text
        note = comment.find('note').text.replace(',', '.')
        commentaire = comment.find('commentaire').text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append({
            'movie': movie,
            'review_id': review_id,
            'name': name,
            'user_id': user_id,
            'note': note,
            'commentaire': commentaire
        })

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df

In [8]:
#fonction pour le fichier de test
def read_xml_to_dataframeTest(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        # Initialisation d'un dictionnaire pour chaque commentaire
        comment_data = {}

        # Extraction de chaque élément si disponible et ajout au dictionnaire
        movie = comment.find('movie')
        if movie is not None:
            comment_data['movie'] = movie.text

        review_id = comment.find('review_id')
        if review_id is not None:
            comment_data['review_id'] = review_id.text

        name = comment.find('name')
        if name is not None:
            comment_data['name'] = name.text

        user_id = comment.find('user_id')
        if user_id is not None:
            comment_data['user_id'] = user_id.text

        # Note is not present in the given data, but if it's sometimes included,
        # you can check for it like this:
        note = comment.find('note')
        if note is not None:
            comment_data['note'] = note.text.replace(',', '.')

        commentaire = comment.find('commentaire')
        if commentaire is not None:
            comment_data['commentaire'] = commentaire.text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append(comment_data)

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df

# Récupération des données dans des dataFrame

In [9]:
#création DataFrame
data_dev   = read_xml_to_dataframe(file_dev)
data_train = read_xml_to_dataframe(file_train)
data_test  = read_xml_to_dataframeTest(file_test)

data_train['commentaire'] = data_train['commentaire'].fillna('')
data_dev['commentaire'] = data_dev['commentaire'].fillna('')
data_test['commentaire'] = data_test['commentaire'].fillna('')

In [10]:
data_train['note'] = pd.to_numeric(data_train['note'], errors='coerce')
data_dev['note'] = pd.to_numeric(data_dev['note'], errors='coerce')
data_train['note'] = data_train['note'].apply(lambda x: x * 2)
data_dev['note'] = data_dev['note'].apply(lambda x: x * 2)

# Prétraitement des Commentaires Textuels avec Keras : Tokenisation et Remplissage

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Initialiser le tokenizer de Keras
tokenizer = Tokenizer(num_words=5000)  
tokenizer.fit_on_texts(data_train['commentaire'])

# Convertir les commentaires en séquences d'entiers
X_train = tokenizer.texts_to_sequences(data_train['commentaire'])
X_test = tokenizer.texts_to_sequences(data_test['commentaire'])

# Définir une longueur maximale
MAX_LENGTH = 500

# Padding et troncature des vecteurs
X_train_padded = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [ ]:
# Affichage du résultat
X_train_padded.shape

(665962, 500)

In [ ]:
Y_train  = data_train['note']
y_train_adjusted = Y_train - 1
y_train_adjusted.shape

(665962,)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jan  6 13:51:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Réseau de neurones convolutif  

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# Vérifier si CUDA (GPU) est disponible et le définir comme device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class CommentClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super(CommentClassifierCNN, self).__init__()

        # Exemple de couches
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=5)
        self.batchnorm = nn.BatchNorm1d(num_features=128)
        self.fc = nn.Linear(12544, num_classes)  # Ajustement de la dimension d'entrée
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.unsqueeze(1)  # Ajout d'une dimension de canal
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.batchnorm(x)
        x = x.view(x.size(0), -1)  # Redimensionnement pour les couches linéaires
        x = self.fc(x)
        return x


num_classes = 10
model = CommentClassifierCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train = X_train_padded
y_train = y_train_adjusted

batch_size = 32
num_epochs = 60

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')


Using device: cuda
Epoch 1/60, Loss: 2.2010
Epoch 2/60, Loss: 2.1959
Epoch 3/60, Loss: 2.1937
Epoch 4/60, Loss: 2.1910
Epoch 5/60, Loss: 2.1889
Epoch 6/60, Loss: 2.1878
Epoch 7/60, Loss: 2.1889
Epoch 8/60, Loss: 2.1870
Epoch 9/60, Loss: 2.1870
Epoch 10/60, Loss: 2.1868
Epoch 11/60, Loss: 2.1863
Epoch 12/60, Loss: 2.1861
Epoch 13/60, Loss: 2.1860
Epoch 14/60, Loss: 2.1862
Epoch 15/60, Loss: 2.1856
Epoch 16/60, Loss: 2.1853
Epoch 17/60, Loss: 2.1848
Epoch 18/60, Loss: 2.1841
Epoch 19/60, Loss: 2.1839
Epoch 20/60, Loss: 2.1830
Epoch 21/60, Loss: 2.1823
Epoch 22/60, Loss: 2.1817
Epoch 23/60, Loss: 2.1811
Epoch 24/60, Loss: 2.1808
Epoch 25/60, Loss: 2.1801
Epoch 26/60, Loss: 2.1805
Epoch 27/60, Loss: 2.1797
Epoch 28/60, Loss: 2.1793
Epoch 29/60, Loss: 2.1794
Epoch 30/60, Loss: 2.1794
Epoch 31/60, Loss: 2.1790
Epoch 32/60, Loss: 2.1786
Epoch 33/60, Loss: 2.1784
Epoch 34/60, Loss: 2.1778
Epoch 35/60, Loss: 2.1776
Epoch 36/60, Loss: 2.1775
Epoch 37/60, Loss: 2.1774
Epoch 38/60, Loss: 2.1772
Ep

# Tester le modèle

In [ ]:
# Convertir les données de test en tenseurs PyTorch
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.float32)

# Créer un DataLoader pour les données de test
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Passer le modèle en mode évaluation
model.eval()

# Faire des prédictions
predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch[0].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())


In [ ]:
# Convertir les données de test en tenseurs PyTorch
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.float32)

# Passer le modèle en mode évaluation
model.eval()

# Faire des prédictions
predictions = []
with torch.no_grad():
    for i in range(len(X_test_tensor)):
        outputs = model(X_test_tensor[i].unsqueeze(0))
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(predicted.item())

In [ ]:
# Convertir les prédictions en une série Pandas
predictions_series = pd.Series(predictions)
predictions_series

0        6
1        7
2        7
3        7
4        7
        ..
85842    7
85843    7
85844    7
85845    7
85846    7
Length: 85847, dtype: int64

In [ ]:
# Ajuster les prédictions
adjusted_predictions = (predictions_series + 1) / 2
adjusted_predictions

0        3.5
1        4.0
2        4.0
3        4.0
4        4.0
        ... 
85842    4.0
85843    4.0
85844    4.0
85845    4.0
85846    4.0
Length: 85847, dtype: float64

In [ ]:
# Ajouter les prédictions ajustées comme une nouvelle colonne à data_test
data_test['note'] = adjusted_predictions
dataTest = data_test

In [ ]:
data_test = dataTest

In [ ]:
import pandas as pd

# Fonction pour convertir un nombre flottant en chaîne avec une virgule pour le décimal
def float_to_comma_string(x):
    if isinstance(x, float):
        return '{:.1f}'.format(x).replace('.', ',')
    return x

# Convertir les notes (prédictions) en nombres
#data_test['note'] = pd.to_numeric(data_test['note'], errors='coerce')
data_test['note'] = data_test['note'].apply(float_to_comma_string)
data_test['note']

0        3,5
1        4,0
2        4,0
3        4,0
4        4,0
        ... 
85842    4,0
85843    4,0
85844    4,0
85845    4,0
85846    4,0
Name: note, Length: 85847, dtype: object

In [ ]:
# Fusionner les prédictions avec review_id
df_test = pd.concat([data_test['review_id'], data_test['note']], axis=1)

# Enregistrer dans un nouveau fichier texte séparé par un espace
df_test.to_csv('/content/drive/My Drive/Analyse_des_sentiment/data/test_outputRnn.txt', sep=' ', index=False, header=False)

# TF_IDF

In [ ]:
import pickle
with open('/content/drive/My Drive/Analyse_des_sentiment/data/dataTF/tfidf_dev.pkl', 'rb') as f:
    X_dev_padded  = pickle.load(f)

with open('/content/drive/My Drive/Analyse_des_sentiment/data/dataTF/tfidf_train.pkl', 'rb') as f:
     X_train_padded = pickle.load(f)

with open('/content/drive/My Drive/Analyse_des_sentiment/data/dataTF/tfidf_test.pkl', 'rb') as f:
    X_test_padded  = pickle.load(f)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Création de l'objet TruncatedSVD avec 900 composantes
svd = TruncatedSVD(n_components=900)

# Réduction de la dimensionnalité
X_train_reduced = svd.fit_transform(X_train_padded)
X_dev_reduced  = svd.fit_transform(X_test_padded)
X_test_reduced = svd.fit_transform(X_test_padded)

In [ ]:
Y_train  = data_train['note']
y_train_adjusted = Y_train - 1
y_train_adjusted.shape

(665962,)

In [ ]:
Y_dev  = data_dev['note']
y_dev_adjusted = Y_dev - 1
y_dev_adjusted.shape

(100400,)

In [ ]:
print(X_train_reduced.shape)
print(X_dev_reduced.shape)
print(X_test_reduced.shape)

(665962, 900)
(85847, 900)
(85847, 900)


In [ ]:
y_train = y_train_adjusted
y_dev = y_dev_adjusted

In [ ]:
class CommentClassifierCNN(nn.Module):
    def __init__(self, num_classes, sample_size):
        super(CommentClassifierCNN, self).__init__()

        # Couches convolutives et pooling
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=5)
        self.batchnorm = nn.BatchNorm1d(num_features=128)

        # Calcul dynamique de la taille d'entrée pour la couche linéaire
        self._to_linear = None
        self._sample_size = sample_size
        self._init_to_linear()

        # Couche linéaire
        self.fc = nn.Linear(self._to_linear, num_classes)
        self.relu = nn.ReLU()

    def _init_to_linear(self):
        # Utiliser un échantillon pour calculer la taille de sortie de la couche convolutive/pooling
        sample_data = torch.zeros(1, 1, self._sample_size)
        sample_data = self.conv1(sample_data)
        sample_data = self.conv2(sample_data)
        sample_data = self.pool(sample_data)
        sample_data = self.batchnorm(sample_data)
        self._to_linear = sample_data.numel()

    def forward(self, x):
        # Mêmes opérations que précédemment
        x = x.unsqueeze(1)  # Ajout d'une dimension de canal
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.batchnorm(x)
        x = x.view(x.size(0), -1)  # Redimensionnement pour les couches linéaires
        x = self.fc(x)
        return x

# Utilisation du modèle
sample_size = X_train.shape[1]  # Remplacer par la taille réelle de vos embeddings
model = CommentClassifierCNN(num_classes, sample_size).to(device)

num_classes = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train = X_train_reduced
y_train = y_train_adjusted

batch_size = 32
num_epochs = 60

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

Epoch 1/60, Loss: 1.7503
Epoch 2/60, Loss: 1.6832
Epoch 3/60, Loss: 1.6702
Epoch 4/60, Loss: 1.6676
Epoch 5/60, Loss: 1.6633
Epoch 6/60, Loss: 1.6612
Epoch 7/60, Loss: 1.6557
Epoch 8/60, Loss: 1.6483
Epoch 9/60, Loss: 1.6429
Epoch 10/60, Loss: 1.6394
Epoch 11/60, Loss: 1.6393
Epoch 12/60, Loss: 1.6364
Epoch 13/60, Loss: 1.6346
Epoch 14/60, Loss: 1.6305
Epoch 15/60, Loss: 1.6276
Epoch 16/60, Loss: 1.6238
Epoch 17/60, Loss: 1.6204
Epoch 18/60, Loss: 1.6174
Epoch 19/60, Loss: 1.6155
Epoch 20/60, Loss: 1.6133
Epoch 21/60, Loss: 1.6112
Epoch 22/60, Loss: 1.6096
Epoch 23/60, Loss: 1.6077
Epoch 24/60, Loss: 1.6057
Epoch 25/60, Loss: 1.6045
Epoch 26/60, Loss: 1.6035
Epoch 27/60, Loss: 1.6025
Epoch 28/60, Loss: 1.6012
Epoch 29/60, Loss: 1.6002
Epoch 30/60, Loss: 1.5997
Epoch 31/60, Loss: 1.5981
Epoch 32/60, Loss: 1.5971
Epoch 33/60, Loss: 1.5959
Epoch 34/60, Loss: 1.5944
Epoch 35/60, Loss: 1.5938
Epoch 36/60, Loss: 1.5932
Epoch 37/60, Loss: 1.5926
Epoch 38/60, Loss: 1.5918
Epoch 39/60, Loss: 1.

# Test

In [ ]:
X_test = X_test_reduced

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def predict(model, test_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in test_loader:
            inputs = batch[0]
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

predictions = predict(model, test_loader)

In [ ]:
# Ajouter les prédictions ajustées comme une nouvelle colonne à data_test
predictions_series = pd.Series(predictions)
adjusted_predictions = (predictions_series + 1) / 2
data_test['note'] = adjusted_predictions
dataTest = data_test
dataTest
import pandas as pd

# Fonction pour convertir un nombre flottant en chaîne avec une virgule pour le décimal
def float_to_comma_string(x):
    if isinstance(x, float):
        return '{:.1f}'.format(x).replace('.', ',')
    return x

# Convertir les notes (prédictions) en nombres
#data_test['note'] = pd.to_numeric(data_test['note'], errors='coerce')
data_test['note'] = data_test['note'].apply(float_to_comma_string)

# Fusionner les prédictions avec review_id
df_test = pd.concat([data_test['review_id'], data_test['note']], axis=1)

# Enregistrer dans un nouveau fichier texte séparé par un espace
df_test.to_csv('/content/drive/My Drive/Analyse_des_sentiment/data/test_output_CNN_TF_IDF.txt', sep=' ', index=False, header=False)

# Bert-base-uncased

In [1]:
from transformers import BertTokenizer, BertModel
import torch

# Initialisation du tokenizer et du modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Vérifier si un GPU est disponible et configurer le modèle BERT pour le GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
def get_embeddings(texts, max_length=500):
    # Tronquer les textes si leur longueur dépasse max_length
    truncated_texts = [text[:max_length] for text in texts]

    # Préparer les inputs pour le modèle BERT et les déplacer sur le GPU
    inputs = tokenizer(truncated_texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
    inputs = inputs.to(device)

    with torch.no_grad():
        # Exécuter le modèle BERT sur le GPU
        outputs = bert_model(**inputs)

    # Retourner les embeddings en tant que tensor PyTorch
    return outputs.last_hidden_state[:, 0, :]

In [ ]:
# Appliquer la fonction à la colonne de texte du DataFrame
data_train['embeddings'] = data_train['commentaire'].apply(lambda x: get_embeddings([x]))

In [ ]:
data_test['embeddings'] = data_test['commentaire'].apply(lambda x: get_embeddings([x]))

In [ ]:
X_train_padded = data_train['embeddings']
X_test_padded = data_test['embeddings']

In [ ]:
print(X_train_padded)
print(X_test_padded)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# Vérifier si CUDA (GPU) est disponible et le définir comme device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class CommentClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super(CommentClassifierCNN, self).__init__()

        # Exemple de couches
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=5)
        self.batchnorm = nn.BatchNorm1d(num_features=128)
        self.fc = nn.Linear(12544, num_classes)  # Ajustement de la dimension d'entrée
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.unsqueeze(1)  # Ajout d'une dimension de canal
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.batchnorm(x)
        x = x.view(x.size(0), -1)  # Redimensionnement pour les couches linéaires
        x = self.fc(x)
        return x


num_classes = 10
model = CommentClassifierCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train = X_train_padded
y_train = y_train_adjusted

batch_size = 32
num_epochs = 60

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

# Test

In [ ]:
X_test = X_test_padded

X_train_tensor = torch.tensor(X_test, dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def predict(model, test_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in test_loader:
            inputs = batch[0]
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

predictions = predict(model, test_loader)

In [ ]:
# Ajouter les prédictions ajustées comme une nouvelle colonne à data_test
predictions_series = pd.Series(predictions)
adjusted_predictions = (predictions_series + 1) / 2
data_test['note'] = adjusted_predictions
dataTest = data_test
dataTest
import pandas as pd

# Fonction pour convertir un nombre flottant en chaîne avec une virgule pour le décimal
def float_to_comma_string(x):
    if isinstance(x, float):
        return '{:.1f}'.format(x).replace('.', ',')
    return x

# Convertir les notes (prédictions) en nombres
#data_test['note'] = pd.to_numeric(data_test['note'], errors='coerce')
data_test['note'] = data_test['note'].apply(float_to_comma_string)

# Fusionner les prédictions avec review_id
df_test = pd.concat([data_test['review_id'], data_test['note']], axis=1)

# Enregistrer dans un nouveau fichier texte séparé par un espace
df_test.to_csv('/content/drive/My Drive/Analyse_des_sentiment/data/test_output_CNN_Embeding.txt', sep=' ', index=False, header=False)